### Import packages

In [1]:
import numpy as np
import pandas as pd

from generator import fitness, generate
from utility.loader import get_batch
from utility.vae import VAE
from chainer import serializers

d:\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


### Configurations:

In [2]:
file_path = "."
model_dir = "saved_models"

train_path = f"{file_path}/{model_dir}/train.txt"

### Initialize variational autoencoder:

In [3]:
vae = VAE(50, 1, 400, 0.3, 60, 180, train_path, 501)
model_name = f"{file_path}/{model_dir}/models/aibirds_word_500"
serializers.load_npz(model_name, vae)

*   Encode training data into latent vectors:

In [4]:
for batch in get_batch(vae.vocab, train_path, 180, False):
    mu, _ = vae.encode(batch)

In [5]:
latent_vecs = mu.array

*   Save in a dataframe

In [6]:
columns = [f"dim{i}" for i in range(1,61)]
encoded_training = pd.DataFrame(latent_vecs, columns=columns)
encoded_training.head()

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim51,dim52,dim53,dim54,dim55,dim56,dim57,dim58,dim59,dim60
0,0.146544,-0.142479,0.156169,-0.175538,0.302417,-0.071271,-0.012902,-0.027739,-0.114900,-0.136111,...,0.719494,0.201832,-0.117302,0.005195,-0.315903,0.054833,0.236369,1.004022,0.223223,-0.071229
1,0.015102,0.091026,0.139189,-0.257915,-0.223623,-0.094606,-0.014248,-0.114900,-0.135874,0.096594,...,0.811771,0.064126,-0.141807,-0.224444,0.031261,0.021533,0.042979,1.238492,0.380081,-0.125611
2,0.093116,-0.063505,-0.045592,0.149789,-0.091598,0.040074,-0.178543,-0.055285,0.037268,0.024610,...,-0.301993,0.179947,0.025036,-0.066210,-0.229901,0.142224,0.142641,-0.625883,-0.150603,-0.047738
3,-0.017193,-0.068952,-0.070259,-0.220292,0.404467,0.083280,0.238296,0.387524,0.113092,-0.056564,...,-0.895009,0.251918,-0.139248,0.178524,0.197034,-0.113163,-0.256855,0.225072,-0.166616,-0.034359
4,0.063370,-0.058646,-0.240935,-0.312472,0.723495,0.158388,0.249680,0.360090,-0.111475,0.145971,...,0.190381,0.111714,-0.294294,0.179444,-0.213940,0.222174,-0.054278,-1.323820,0.069491,0.053679


*   Save as csv file
    *   `encoded_training.csv` is later moved to other directories.

In [7]:
encoded_training.to_csv("encoded_training.csv", index=False)

*   Obtain fitness

In [8]:
latent_vecs_list = []
for i in range(len(latent_vecs)):
    latent_vecs_list.append(np.reshape(latent_vecs[i], (1,60)))

In [15]:
fitness_list = fitness(latent_vecs = latent_vecs_list, 
                       file_path = file_path, 
                       model_dir = model_dir)

100%|██████████| 180/180 [00:38<00:00,  4.70it/s]


In [18]:
fitness_columns = ['pig_count', 'tnt_count', 'material_types', 'object_types', 'object_count']
encoded_training_fitness = pd.DataFrame(np.array(fitness_list), columns=fitness_columns)
encoded_training_fitness.head()

,pig_count,tnt_count,material_types,object_types,object_count
0,1,0,4,8,19
1,2,3,4,11,70
2,4,0,4,7,41
3,1,1,4,9,21
4,4,0,4,6,19


*   Save as csv file
    *   `encoded_training_fitness.csv` is later moved to other directories.

In [19]:
encoded_training_fitness.to_csv("encoded_training_fitness.csv", index=False)